In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import pdfkit


class ExcelTableReader:
    def __init__(self, url):
        self.url = url
        self.data = None

    def load_excel(self):
        try:
            self.data = pd.read_excel(self.url, sheet_name=None)
            print("Excel file loaded successfully.")
        except Exception as e:
            print(f"Error loading Excel file: {str(e)}")

    def get_sheet_names(self):
        if self.data is not None:
            return list(self.data.keys())
        else:
            print("No Excel file loaded.")
            return []

    def get_table_names(self, sheet_name):
        if sheet_name in self.data.keys():
            sheet_data = self.data[sheet_name]
            table1_name = sheet_data.iloc[3, 6]
            table2_name = sheet_data.iloc[4, 0]
            return table1_name, table2_name
        else:
            print(f"Sheet '{sheet_name}' does not exist.")
            
    def get_table(self, sheet_name, table_name):
        if sheet_name in self.data.keys():
            sheet_data = self.data[sheet_name]
            table_names = self.get_table_names(sheet_name)
            if table_name == table_names[0]:
                return sheet_data.iloc[5:16, 6:13+1]
            elif table_name == table_names[1]:
                return sheet_data.iloc[5:16, 0:4+1]
            else:
                print("Invalid table name.")
        else:
            print(f"Sheet '{sheet_name}' does not exist.")
        
class ExcelDropdown:
    def __init__(self, sheet_names, table_names):
        self.sheet_dropdown = widgets.Dropdown(
            options=sheet_names,
            description="Select sheet:",
        )
        self.table_dropdown = widgets.Dropdown(
            options=table_names,
            description="Select table:",
        )
        self.toggle_button = widgets.ToggleButtons(
            options=["View", "Save"],
            button_style = "primary",
            description="Action:",
        )
        self.button = widgets.Button(button_style="danger",description="Execute")
        self.output = widgets.Output()

    def create_dropdown(self):
        def execute_action(btn):
            selected_sheet = self.sheet_dropdown.value
            selected_table = self.table_dropdown.value
            action = self.toggle_button.value

            if action == "View":
                table = reader.get_table(selected_sheet, selected_table)
                if table is not None:
                    with self.output:
                        self.output.clear_output()
                        display(table)
            elif action == "Save":
                table = reader.get_table(selected_sheet, selected_table)
                if table is not None:
                    table.to_excel(f"{selected_sheet}_{selected_table}.xlsx")
                    print(f"Table saved as {selected_sheet}_{selected_table}.xlsx")

        self.button.on_click(execute_action)
        display(widgets.VBox([self.sheet_dropdown, self.table_dropdown, self.toggle_button, self.button, self.output]))


if __name__ == "__main__":
    url = "https://www.nrel.gov/wind/nwtc/assets/downloads/MetOcean/DistributionParameters.xlsx"
    reader = ExcelTableReader(url)
    reader.load_excel()

    sheet_names = reader.get_sheet_names()
    table_names = reader.get_table_names(sheet_names[0])
    dropdown = ExcelDropdown(sheet_names, table_names)
    dropdown.create_dropdown()

Excel file loaded successfully.
